1. **Dataset Introduction**: Iris and SpamBase datasets are used here. Iris has three classes and the task is to accurately predict one of the three subtypes of the Iris flower given four different physical features. Spambase is a binary classification task and the objective is to classify email messages as being spam or not. There are about 4600 instances

2. **Implementation idea**: to determine best split, searched over all possible thresholds for a given feature and information gran detemrined by entropy is used to determine the best split with the most information gain.  



In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt

In [4]:
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
with pd.option_context('future.no_silent_downcasting', True):
    iris_df= iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2}).infer_objects()

In [33]:
class DecisionTree:
    def __init__(self,nmin):
        self.tree = None
        self.nmin = nmin

    def entropy(self,y):
        probabilities = np.bincount(y) / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def information_gain(self,X,y,feature,threshold):
        parent_entropy = self.entropy(y)
        left_indices = X[:, feature] <= threshold
        right_indices = X[:, feature] > threshold
        if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
            return 0
        n = len(y)
        n_left, n_right = np.sum(left_indices), np.sum(right_indices)
        e_left, e_right = self.entropy(y[left_indices]), self.entropy(y[right_indices])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right
        return parent_entropy - child_entropy

    def most_common_label(self,y):
        unique_labels, counts = np.unique(y, return_counts=True)
        return unique_labels[np.argmax(counts)]

    def best_split(self,X,y):
        # find best feature and threshold
        m,n = X.shape
        best_gain = -1
        best_feature,best_threshold = None,None
        for feature in range(n):
            thresholds = np.unique(X[:,feature])
            for threshold in thresholds: 
                gain = self.information_gain(X,y,feature,threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature,best_threshold

    def fit(self,X,y):
        self.tree = self.grow_tree(X,y)

    def grow_tree(self,X,y):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))
        
        if num_samples < self.nmin or num_labels ==1:
            leaf_value = self.most_common_label(y)
            return leaf_value 
            
        best_feature, best_threshold = self.best_split(X, y)
        if best_feature is None:
            return self.most_common_label(y)
            
        left_indices = X[:,best_feature] <= best_threshold
        right_indices = X[:,best_feature] > best_threshold
        
        left_subtree = self.grow_tree(X[left_indices], y[left_indices])
        right_subtree = self.grow_tree(X[right_indices], y[right_indices])
        
        return (best_feature, best_threshold, left_subtree, right_subtree)

    def predict_sample(self,sample,tree):
        if not isinstance(tree, tuple):  
            return tree
        feature, threshold, left_subtree, right_subtree = tree
        if sample[feature] <= threshold:
            return self.predict_sample(sample, left_subtree)
        else:
            return self.predict_sample(sample, right_subtree)

    def predict(self,X):
        return [self.predict_sample(x, self.tree) for x in X]

In [56]:
def evaluate_model(X, y, nmin_values):
    results = {}
    for nmin in nmin_values:
        accuracies = []
        print(f"Evaluating nmin={nmin}...") 
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            tree = DecisionTree(nmin=nmin)
            tree.fit(X_train, y_train)            
            y_pred = tree.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)

        results[nmin] = (np.mean(accuracies), np.std(accuracies))
    return results

In [35]:
print("Evaluating Iris dataset...")

Evaluating Iris dataset...


In [57]:
X_iris = iris_df.iloc[:, :-1].values  
y_iris = iris_df.iloc[:, -1].values  
nmin_iris = [5, 10, 15, 20]

In [58]:
iris_results = evaluate_model(X_iris, y_iris, nmin_iris)
for nmin, (mean_acc, std_acc) in iris_results.items():
    print(f"nmin={nmin}: Accuracy = {mean_acc:.4f} ± {std_acc:.4f}")

Evaluating nmin=5...
Evaluating nmin=10...
Evaluating nmin=15...
Evaluating nmin=20...
nmin=5: Accuracy = 0.9333 ± 0.0516
nmin=10: Accuracy = 0.9467 ± 0.0499
nmin=15: Accuracy = 0.9467 ± 0.0499
nmin=20: Accuracy = 0.9467 ± 0.0499


In [55]:
spam_df = pd.read_csv('spambase.csv') 
nmin_spam= [5, 10, 15, 20,25]
X_spam = spam_df.iloc[:, :-1].values  
y_spam = spam_df.iloc[:, -1].values  

         0  0.64  0.64.1  0.1  0.32   0.2   0.3   0.4   0.5   0.6  ...   0.41  \
0     0.21  0.28    0.50  0.0  0.14  0.28  0.21  0.07  0.00  0.94  ...  0.000   
1     0.06  0.00    0.71  0.0  1.23  0.19  0.19  0.12  0.64  0.25  ...  0.010   
2     0.00  0.00    0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  ...  0.000   
3     0.00  0.00    0.00  0.0  0.63  0.00  0.31  0.63  0.31  0.63  ...  0.000   
4     0.00  0.00    0.00  0.0  1.85  0.00  0.00  1.85  0.00  0.00  ...  0.000   
...    ...   ...     ...  ...   ...   ...   ...   ...   ...   ...  ...    ...   
4595  0.31  0.00    0.62  0.0  0.00  0.31  0.00  0.00  0.00  0.00  ...  0.000   
4596  0.00  0.00    0.00  0.0  0.00  0.00  0.00  0.00  0.00  0.00  ...  0.000   
4597  0.30  0.00    0.30  0.0  0.00  0.00  0.00  0.00  0.00  0.00  ...  0.102   
4598  0.96  0.00    0.00  0.0  0.32  0.00  0.00  0.00  0.00  0.00  ...  0.000   
4599  0.00  0.00    0.65  0.0  0.00  0.00  0.00  0.00  0.00  0.00  ...  0.000   

       0.42  0.43  0.778   

In [53]:
spam_result = evaluate_model(X_spam,y_spam,nmin_spam)
print(spam_result)
for nmin,(mean_acc,std_acc) in spam_result.items():
    print(f"nmin={nmin}: Accuracy = {mean_acc:.4f} ± {std_acc:.4f}")

Evaluating nmin=5...
Evaluating nmin=10...
Evaluating nmin=15...
Evaluating nmin=20...
Evaluating nmin=25...
{5: (0.9189130434782609, 0.01573798938933761), 10: (0.9173913043478261, 0.01552484440987574), 15: (0.9197826086956521, 0.01625209016657884), 20: (0.9202173913043479, 0.016671234408728565), 25: (0.9204347826086956, 0.015921594281391796)}
nmin=5: Accuracy = 0.9189 ± 0.0157
nmin=10: Accuracy = 0.9174 ± 0.0155
nmin=15: Accuracy = 0.9198 ± 0.0163
nmin=20: Accuracy = 0.9202 ± 0.0167
nmin=25: Accuracy = 0.9204 ± 0.0159
